#### Общая задача: 

 
Предположим что существует видео хостинг на котором загружено большое количество видео. Большая часть видео не имеет нормального названия, описания, тегов. Видео предполагают тематику 18+. Ваша задача расписать алгоритм действий, как создать инструмент который бы генерировал такие параметры для видео на основе их содержания:

 
- Описание видео (предполагая что это описание можно будет разместить на сайте и оно будет удобно читаться человеком) - должно вписываться в тематику 18+ сайтов;

- Теги для видео 18+;

- Название для видео которое хорошо бы отображало суть видео и вписывалось бы в тематику 18+ сайтов;

- Весь текст будет на английском языке.

 
##### Что допустимо в алгоритме действий:

 
- использование уже готовых нейросетей - можно найти готовую нейросеть и дообучить ее для использования в Adult тематике;

- использование серверов компании для вычислений и обучения нейросети;

- использовать готовый датасет, уже присутствует определенное количество видео для которых сделаны описания, теги, названия вручную;

- написание своей нейросети для этой задачи;

- комбинирование разных нейросетей, например object detection/image2text/video2text + text2text нейросетей для получения более качественных текстовых результатов;

- можете добавить свой вариант который вы считаете более оптимальным для этой задачи.

 
##### Сложности с которыми вы можете столкнуться:

 
- готовые нейросети не предназначены для 18+ тематики и вы можете столкнуться с тем что они имеют блокировки для обработки запрещенных слов.

 
#### Что нужно сделать в тестовой задаче:

 
- расписать алгоритм действий, желательно подробно, как бы вы подошли к этой задаче;

- описать алгоритм подготовки датасетов и требования к ним;

- описать сложности с которыми вы можете столкнуться и пути их решения;

- описать необходимые ресурсы которые могут потребоваться для решения этой задачи;

- готовую нейросеть предоставлять не нужно, только алгоритм действий.

 

#### 1 этап

1. Протестировать модель YOLO для получения названий ключевых объектов на видео 18+

1.2 В случае неудовлетворительных результатов (плохая детекция органов и тд):

 1.2.1 Применить модель

https://github.com/vladmandic/nudenet

для создания датасета с объектами sexual body parts

 - Разбить видео на кадры и подайте их на вход модели.
 
 - Записать результаты (координаты объектов, их классы).
 
1.2.2 Дообучить модель YOLO на новом датасете


Возможно будут полезны следующие датасеты:

https://www.kaggle.com/datasets/brijeshbmehta/adult-datasets?resource=download

https://huggingface.co/datasets/deepghs/nsfw_detect

Для увеличения количества данных в датасете можно применить аугментации.

#### 2 этап - создание текстовых датасетов

Применить ChatGPT (GPT3, GPT4) для генерации отличных описаний, тегов и названий

Сложность: фильтры и политика ChatGPT не позволят сгенерировать такие данные для контента 18+

Решение: использование локальных LLM

 https://github.com/nomic-ai/gpt4all
 
 https://github.com/imartinez/privateGPT
 
 для задач создания датасетов dataset_descriptions, dataset_tags, dataset_names, используя prompt engineering

Чтобы увеличить размеры и разнообразие датасетов: dataset_descriptions, dataset_tags, dataset_names
Нужно применить аугментацию путем prompt engineering (просить сгенерить сразу несколько вариаций описаний, тегов и названий).

#### 3 этап - обучение моделей для генераций описаний, тегов и названий для видео 18+

##### 1 сценарий:

Генерация описания

Модель: T5

Датасет: dataset_descriptions

    входные данные - это названия ключевых объектов из видео
    
    выходные данные - полные описания видео на основе этих слов
    
Дообучить на новом датасете модель T5 [t5_description]

Генерация тегов

Модель: T5

Датасет: dataset_tags

    входные данные - полные описания видео
    
    выходные данные - теги видео
    
Дообучить на новом датасете модель T5 [t5_tags]


Генерация названия

Модель: T5

Датасет: dataset_names

    входные данные - полные описания видео
    
    выходные данные - название (заголовок) видео
    
Дообучить на новом датасете модель T5 [t5_name]

##### 2 сценарий:

Можно попробовать использовать одну модель T5, отделять description, tag, name токенами
соотвественно датасет нужен будет такой:

*входные данные*: названия ключевых объектов из видео

*выходные данные*: <\description>description<\description><\tags>tags<\tags><\name>name<\name>

Таким образом у нас будет две модели:

1 модель - извлекает ключевые названия объектов из видео

2 модель - генерирует описание, теги, название для ключевых названий из предыдущего шага


**Необходимые ресурсы**: GeForce RTX 3090 ti или p40 или A100

Эти карточки вместят модель T5-large (770M), тем более YOLOv7 (36.9M)

Большее количество карточек потребуется для более быстрого обучения (Возможно понадобится использование кластеров)